# Client system operations agent system

Este proyecto realiza operaciones sobre los clientes de una empresa. 
En este caso, consulta los clientes de una empresa y genera un e-mail utilizando el idioma prefrido del cliente

1. Utiliza Langchain agents.
1. No requiere suscriciones, Utiliza solamente modelos de LLM locales y ligeros, en este caso usamos qwen3:7b sobre ollama.
1. No requiere conocimientos de IA tooling ya que utiliza el LLM local para generar su própio código python.
1. Utiliza el propio LLM local para generar los datos dummy con los que trabajan el resto de agentes.


In [4]:
%%capture --no-stderr
%pip install -U --upgrade pip langgraph langchain_ollama pandas

In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
workfolder = os.getenv('WORKFOLDER')
llm_config = {"recursion_limit": 10}

In [6]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[], response_format='json', prompt="You are a helpful assistant.")

input_message = {"role": "user", "content": """
Generate the python code for a function that writes a string content in a file. 
a langchain tool with Annotated parameters and return type.
3 parameters: content, file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a 30 words docstring
use encoding utf8
give just the code, no introduction /no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


Generate the python code for a function that writes a string content in a file. 
a langchain tool with Annotated parameters and return type.
3 parameters: content, file_path and append (True or False)
should be able to handle exceptions and return an error message if the file cannot be written.
Function must have a 30 words docstring
use encoding utf8
give just the code, no introduction /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

```python
from langchain.tools import tool
from typing import Annotated

@tool
def write_to_file(
    content: Annotated[str, "The content to write to the file."],
    file_path: Annotated[str, "The path to the file."],
    append: Annotated[bool, "Whether to append to the file or overwrite it."]
) -> Annotated[str, "Result message indicating success or error."]:
    """
    Writes content to a

In [13]:
from langchain.tools import tool
from typing import Annotated, Optional

@tool
def write_file(
    content: Annotated[str, "The content to be written to the file."],
    file_path: Annotated[str, "The path to the file where content will be written."],
    append: Annotated[bool, "If True, append to the file; if False, overwrite it."],
    encoding: Annotated[Optional[str], "The encoding of the file."] = 'utf-8'
) -> Annotated[str, "Result message indicating success or error."]:
    """Writes content to a file in append or overwrite mode."""
    try:
        mode = 'a' if append else 'w'
        content = '\n'+content if append else content
        with open(file_path, mode, encoding=encoding) as f:
            f.write(content)
        return f"Content successfully written to {file_path}"
    except Exception as e:
        return f"Error writing to file: {str(e)}"

write_file.invoke(input={'content':'this is a test\n', 'file_path':'/tmp4/file.txt', 'append': True})

"Error writing to file: [Errno 2] No such file or directory: '/tmp4/file.txt'"

In [15]:
simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[write_file], response_format='json', prompt="You are a helpful assistant.")

input_message = {"role": "user", "content": f"""
generate a csv list of 10 dummy clients with the following columns:
id, name, address, phone, email, prefered_languaje
write the list to file in '{workfolder}/clients.csv'
 /no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


generate a csv list of 10 dummy clients with the following columns:
id, name, address, phone, email, prefered_languaje
write the list to file in '/home/daimler/workspaces/agents-course-huggingface/.workspace/clients.csv'
 /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>
Tool Calls:
  write_file (d8013bdc-9783-4251-ad2f-0606b4538ca3)
 Call ID: d8013bdc-9783-4251-ad2f-0606b4538ca3
  Args:
    append: False
    content: id,name,address,phone,email,prefered_languaje
1,John Doe,123 Main St,555-1234,john@example.com,English
2,Jane Smith,456 Oak St,555-5678,jane@example.com,Spanish
3,Bob Johnson,789 Pine St,555-9012,bob@example.com,French
4,Alice Williams,101 Maple St,555-1357,alice@example.com,German
5,Michael Brown,202 Cedar St,555-2468,michael@example.com,Italian
6,Emily Davis,303 Birch St,555-3579,emily@example.com,Dutch
7,David

In [32]:
input_message = {"role": "user", "content": f"""
Generate the python code for a function that return dict with all the client details for a given id. 
is a langchain tool with typing Annotated .
parameters id int
use pandas to load a csv file in '{workfolder}/clients.csv' in utf8
handle exceptions
Function must have a 20 word docstring
give just the code, no introduction /no_think"""
}

# Use the agent
for step in simple_agent.stream({"messages": [input_message]}, llm_config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================


Generate the python code for a function that return dict with all the client details for a given id. 
is a langchain tool with typing Annotated .
parameters id int
use pandas to load a csv file in '/home/daimler/workspaces/agents-course-huggingface/.workspace/clients.csv' in utf8
handle exceptions
Function must have a 20 word docstring
give just the code, no introduction /no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

from langchain.tools import tool
import pandas as pd
from typing import Annotated, Dict

@tool
def get_client_details(id: Annotated[int, "Client ID"]) -> Dict[str, str]:
    """Fetches client details by ID from a CSV file. Handles exceptions and returns a dictionary."""
    try:
        file_path = "/home/daimler/workspaces/agents-course-huggingface/.workspace/clients.csv"
        df = pd.read_csv(file_path, e

In [7]:
import pandas as pd
from langchain_core.tools import tool 

@tool
def get_client_details(client_id):
    """
    Retrieve client details of the provided client ID.

    Parameters:
    client_id (str): The ID of the client to retrieve details for.

    Returns:
    dict: A dictionary containing all client details if found, otherwise an error message.
    """
    try:
        file_path = workfolder+'/clients.csv'
        df = pd.read_csv(file_path, encoding='utf-8')
        client_details = df[df['client_id'] == int(client_id)].to_dict(orient='records')
        if client_details:
            return client_details[0]
        else:
            return {"error": "Client not found"}
    except FileNotFoundError:
        return {"error": "File not found"}
    except Exception as e:
        return {"error": str(e)}

get_client_details.invoke('31')

{'client_id': 31,
 'name': 'Mary Brown',
 'address': '112 Pine St',
 'phone_number': '555-5544',
 'email': 'mary@example.com',
 'prefered_languaje': 'Japanese'}

In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
import random

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=[get_client_details], response_format='json', prompt="You are a helpful assistant. Your task is to respond to a user's question.")

input_message = {"role": "user", "content": f"""
get the client details of a client with id '{random.randint(0, 10)}'
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think"""
}

# Use the agent
for step in simple_agent.stream(
    {"messages": [input_message]}, llm_config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


get the client details of a client with id '11'
generate an email explaining we could not deliver the package at his address nor contact him at his phone number.
use they prefered language
especify the actual address and phone number of the client.
/no_think
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>
Tool Calls:
  get_client_details (3a9ac96e-b82f-4e9e-84d6-42af5f92e2b3)
 Call ID: 3a9ac96e-b82f-4e9e-84d6-42af5f92e2b3
  Args:
    client_id: 11
================================= Tool Message =================================
Name: get_client_details

{"client_id": 11, "name": "Robert Smith", "address": "345 Maple St", "phone_number": "555-6677", "email": "robert@example.com", "prefered_languaje": "Russian"}
================================== Ai Message ==================================
Name: simple_agent

<think>

</think>

Subject